# The Music of the Sphere: Fourier Series Representation of the Horizon-scale Potential

*Phil Marshall and Roger Blandford, August 2015*

In this notebook we look at some simulated potential fields, and predict the CMB temperature map from them.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import beatbox
import healpy as hp

/Users/pjm/lsst/DarwinX86/anaconda/2.1.0-4-g35ca374/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


## 1. Generating and Visualizing a 3D Potential

Our plan is to use a simple Fourier series, with terms lying within some sphere in k-space, to represent the potential on horizon scales. Let's instantiate such a "beatbox" universe and generate a Gaussian random potential field in it.

In [3]:
We = beatbox.Universe()
We.generate_a_random_potential_field(nmax=6)

Generated  924  potential Fourier coefficients
 Built potential grid, with dimensions  (41, 41, 41)  and mean value  0.0 +/- 31.0


Now, let's visualize the 3D potentials, with `yt`. We're trying to make a plot like [this one](http://yt-project.org/doc/_images/vr_sample.jpg), as shown in the [volume rendering tutorial](http://yt-project.org/doc/visualizing/volume_rendering.html):

In [ ]:
We.show_potential_with_yt()

It's true. The code is really simple, [check it out](https://github.com/drphilmarshall/Music/blob/master/beatbox/universe.py#L132) - but we need to concentrate on inferring the potential right now. Go on - submit us a pull request! :-)

## 2. Verifying that Phil's Python agrees with Roger's Mathematica code

This is [issue 8](https://github.com/rogerblandford/Music/issues/8).

Now, convert the Fourier coefficients `fn` into `alm`'s, using the appropriate response matrix.


In [ ]:
We.transform_3D_potential_fourier_series_into_T_spherical_harmonics(lmax=3)